In [3]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

In [4]:
x = [1,2,3,4,5,6,7,8,9,10]
y = [3,5,7,9,11,13,15,17,19,21]

X_train = np.array(x).reshape(-1,1)
y_train = np.array(y)

In [7]:
model=Sequential()
model.add(Dense(units=1, input_shape=(1,)))
model.compile(optimizer='sgd', loss='mse', metrics=['mae'])
model.fit(X_train, y_train, epochs=10)

C:\Users\Hane\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\core\dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - loss: 45.2523 - mae: 6.1630
Epoch 2/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step - loss: 2.2219 - mae: 1.4592
Epoch 3/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step - loss: 0.2464 - mae: 0.4511
Epoch 4/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step - loss: 0.1545 - mae: 0.3244
Epoch 5/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 118ms/step - loss: 0.1491 - mae: 0.3218
Epoch 6/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step - loss: 0.1477 - mae: 0.3204
Epoch 7/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step - loss: 0.1464 - mae: 0.3195
Epoch 8/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step - loss: 0.1452 - mae: 0.3183
Epoch 9/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step - loss: 0.1440 - mae: 0.3170
Epoch 10/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step - loss: 0.1428 - mae: 0.3156


In [8]:
model.layers[0].weights

[<Variable path=sequential_2/dense/kernel, shape=(1, 1), dtype=float32, value=[[2.116742]]>,
 <Variable path=sequential_2/dense/bias, shape=(1,), dtype=float32, value=[0.1872616]>]

# 4고객 만족 분류 모델 구현 실습

## EDA

In [27]:
import pandas as pd
import matplotlib.pyplot as plt

In [25]:
df_original = pd.read_csv('./FLight Price Prediction/Invistico_Airline.csv')
df = df_original.copy()
pd.set_option('display.max_columns', None)
df.head(3)

,satisfaction,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Seat comfort,Departure/Arrival time convenient,Food and drink,Gate location,Inflight wifi service,Inflight entertainment,Online support,Ease of Online booking,On-board service,Leg room service,Baggage handling,Checkin service,Cleanliness,Online boarding,Departure Delay in Minutes,Arrival Delay in Minutes
0,satisfied,Female,Loyal Customer,65,Personal Travel,Eco,265,0,0,0,2,2,4,2,3,3,0,3,5,3,2,0,0.0
1,satisfied,Male,Loyal Customer,47,Personal Travel,Business,2464,0,0,0,3,0,2,2,3,4,4,4,2,3,2,310,305.0
2,satisfied,Female,Loyal Customer,15,Personal Travel,Eco,2138,0,0,0,3,2,0,2,2,3,3,4,4,4,2,0,0.0


In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 129880 entries, 0 to 129879
Data columns (total 23 columns):
 #   Column                             Non-Null Count   Dtype  
---  ------                             --------------   -----  
 0   satisfaction                       129880 non-null  object 
 1   Gender                             129880 non-null  object 
 2   Customer Type                      129880 non-null  object 
 3   Age                                129880 non-null  int64  
 4   Type of Travel                     129880 non-null  object 
 5   Class                              129880 non-null  object 
 6   Flight Distance                    129880 non-null  int64  
 7   Seat comfort                       129880 non-null  int64  
 8   Departure/Arrival time convenient  129880 non-null  int64  
 9   Food and drink                     129880 non-null  int64  
 10  Gate location                      129880 non-null  int64  
 11  Inflight wifi service              1298

## 데이터 전처리

In [36]:
#1. 결측치 - Arrival Delay in Minutes - 평균값

temp = df['Arrival Delay in Minutes'].mean()
df[df['Arrival Delay in Minutes'].isnull()] = temp

In [49]:
#2. object -> string
cols = ['satisfaction','Gender','Customer Type','Type of Travel','Class']
df[cols] = df[cols].astype(str)

In [50]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 129880 entries, 0 to 129879
Data columns (total 23 columns):
 #   Column                             Non-Null Count   Dtype  
---  ------                             --------------   -----  
 0   satisfaction                       129880 non-null  object 
 1   Gender                             129880 non-null  object 
 2   Customer Type                      129880 non-null  object 
 3   Age                                129880 non-null  float64
 4   Type of Travel                     129880 non-null  object 
 5   Class                              129880 non-null  object 
 6   Flight Distance                    129880 non-null  float64
 7   Seat comfort                       129880 non-null  float64
 8   Departure/Arrival time convenient  129880 non-null  float64
 9   Food and drink                     129880 non-null  float64
 10  Gate location                      129880 non-null  float64
 11  Inflight wifi service              1298